In [1]:
import duckdb
import os
import pyarrow.parquet as pq

def convert_file(file_path, output_folder, output_format='duckdb', file_name=None, table_name=None, json_options=None, csv_options=None):
    try:
        # Ensure output folder exists
        os.makedirs(output_folder, exist_ok=True)
        
        # Determine file extension and output file paths
        file_ext = os.path.splitext(file_path)[1].lower()
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        file_name = file_name or f"{base_name}.{output_format}"
        output_file_path = os.path.join(output_folder, file_name)
        table_name = table_name or base_name

        # Use an on-disk DuckDB file to minimize memory usage
        duckdb_file_path = os.path.join(output_folder, f"{base_name}.duckdb")
        conn = duckdb.connect(database=duckdb_file_path)

        # Default CSV/JSON options if not provided
        csv_options = csv_options or {}
        json_options = json_options or {}

        # Handling CSV files
        if file_ext == '.csv':
            # Apply CSV options
            sample_size = csv_options.get('sample_size', 1000000)
            dateformat = csv_options.get('dateformat', '%m/%d/%Y')
            all_varchar = csv_options.get('all_varchar', False)
            column_types = csv_options.get('types', {})

            # Create a table by reading the CSV file in a streaming way
            conn.execute(f"""
                CREATE TABLE {table_name} AS 
                SELECT * FROM read_csv_auto('{file_path}', sample_size={sample_size}, 
                dateformat='{dateformat}', all_varchar={all_varchar}, types={column_types})
            """)

        # Handling Parquet files
        elif file_ext == '.parquet':
            # Create a table from the Parquet file
            conn.execute(f"CREATE TABLE {table_name} AS SELECT * FROM read_parquet('{file_path}')")

        # Handling JSON files
        elif file_ext == '.json':
            format_type = json_options.get('format', 'auto')
            records = json_options.get('records', 'auto')

            # Create a table from the JSON file
            conn.execute(f"CREATE TABLE {table_name} AS SELECT * FROM read_json('{file_path}', format='{format_type}', records='{records}')")

        # Export to the desired format using DuckDB's streaming COPY command
        if output_format == 'parquet':
            # Stream the table to a Parquet file using DuckDB's COPY
            conn.execute(f"COPY (SELECT * FROM {table_name}) TO '{output_file_path}' (FORMAT PARQUET)")

        elif output_format == 'csv':
            # Stream the table to a CSV file using DuckDB's COPY
            conn.execute(f"COPY (SELECT * FROM {table_name}) TO '{output_file_path}' (FORMAT CSV, HEADER)")

        elif output_format == 'json':
            # Stream the table to a JSON file (export using pandas for JSON support)
            df = conn.execute(f"SELECT * FROM {table_name}").fetchdf()
            df.to_json(output_file_path, orient='records', lines=True)

        elif output_format == 'duckdb':
            # The table is already stored in the DuckDB file, no need for further actions
            print(f"DuckDB file created at {duckdb_file_path}")

        print(f"File successfully converted to {output_format} at {output_file_path}")

    except Exception as e:
        print(f"An error occurred during file conversion: {e}")
        print("Troubleshooting steps:")
        print(f"1. Ensure that the file at {file_path} exists.")
        print(f"2. Check if the file formatting is correct.")
        print(f"3. Try adjusting the options like `format`, `records`, or column types.")
    finally:
        conn.close()




In [4]:
# Define the file path and output folder
file_path = '/home/christianocean/quiv/nfl.parquet'
output_folder = '/home/christianocean/evidencenfl/data'

# Specify custom file and table names
custom_file_name = 'nfl'
custom_table_name = 'pbp'

# Call the convert_file function with custom file and table names
convert_file(
    file_path=file_path, 
    output_folder=output_folder, 
    output_format='parquet', 
    file_name=custom_file_name,  # Specify the name for the DuckDB file
    table_name=custom_table_name  # Specify the name for the table
)


DuckDB file created at /home/christianocean/evidencenfl/data/nfl.duckdb
File successfully converted to duckdb at /home/christianocean/evidencenfl/data/nfl


In [3]:
import polars as pl
import pyarrow.parquet as pq

def process_in_chunks(parquet_file: str, date_columns: list, output_parquet_file: str, date_format: str = None, chunk_size: int = 10000):
    # Initialize an empty DataFrame to store results
    processed_df = pl.DataFrame()

    # Open the Parquet file using pyarrow
    parquet_reader = pq.ParquetFile(parquet_file)

    # Process file in chunks
    for batch in parquet_reader.iter_batches(batch_size=chunk_size):
        # Convert pyarrow Table to Polars DataFrame
        df_chunk = pl.from_arrow(batch)

        # Process the chunk
        for col in date_columns:
            if date_format:
                df_chunk = df_chunk.with_columns(
                    pl.col(col).str.strip_chars(" ").str.strptime(pl.Date, format=date_format, strict=False).alias(col)
                )
            else:
                df_chunk = df_chunk.with_columns(
                    pl.col(col).str.strip_chars(" ").str.strptime(pl.Date, strict=False).alias(col)
                )

        # Concatenate processed chunks to the main DataFrame
        processed_df = processed_df.vstack(df_chunk)

    # Write the final processed DataFrame to a new Parquet file
    processed_df.write_parquet(output_parquet_file)
    print(f"New Parquet file saved at: {output_parquet_file}")

# Path to the input Parquet file
input_parquet_file = '/home/christianocean/oceandatachallengesdemo/data/testing/DOB_Permit_Issuance.parquet'

# Path to the output Parquet file
output_parquet_file = '/home/christianocean/oceandatachallengesdemo/data/testing/DOB_Permit_Issuance_converted_chunks.parquet'

# Specify the date columns to cast
date_columns = ['Filing Date', 'Issuance Date', 'Expiration Date', 'Job Start Date']

# Optionally specify the date format if needed, or leave it as None for default handling
date_format = "%Y-%m-%d"

# Run the chunk-based cast and export process
process_in_chunks(input_parquet_file, date_columns, output_parquet_file, date_format, chunk_size=10000)


: 

In [2]:
import polars as pl
from rich.console import Console
from rich.table import Table

def analyze_parquet(file_path):
    # Load the Parquet file
    df = pl.read_parquet(file_path)

    # Create a console object for rich output
    console = Console()

    # Create a table to display the schema with enhanced colors
    table = Table(title="Parquet Schema", title_style="bold green")

    # Add columns for column names and data types with softer, harmonious colors
    table.add_column("Column Name", justify="left", style="bold yellow", no_wrap=True)
    table.add_column("Data Type", justify="left", style="bold cyan")

    # Add rows to the table
    for col_name, col_dtype in df.schema.items():
        table.add_row(col_name, str(col_dtype), style="white on black")

    # Print the schema table
    console.print(table)

    # Print the number of rows
    console.print(f"[bold magenta]\nNumber of rows:[/] [bold white]{df.height}[/]")

# Example usage
file_path = '/home/christianocean/mta/datatest/2024_02/week_4.parquet'
analyze_parquet(file_path)


                          Parquet Schema                          
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Column Name         ┃ Data Type                                ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ fare_class_category │ String                                   │
│ latitude            │ Float64                                  │
│ transit_mode        │ String                                   │
│ station_complex_id  │ String                                   │
│ longitude           │ Float64                                  │
│ station_complex     │ String                                   │
│ payment_method      │ String                                   │
│ ridership           │ Int64                                    │
│ transit_timestamp   │ Datetime(time_unit='us', time_zone=None) │
│ transfers           │ Int64                                    │
│ borough             │ String                                   │
│ geom_wkt            │ String                                   │
└─────────────────────┴──────────────────────────────────────────┘

Number of rows: 423984

In [5]:
import polars as pl
import os
from rich.console import Console
from rich.table import Table

def analyze_parquet_folder(folder_path):
    # Create a console object for rich output
    console = Console()

    # Create a table to display the results
    table = Table(title="Parquet File Summary", title_style="bold green")

    # Add columns for file name, row count, min timestamp, and max timestamp
    table.add_column("File Name", justify="left", style="bold yellow", no_wrap=True)
    table.add_column("Number of Rows", justify="right", style="bold cyan")
    table.add_column("Min Timestamp", justify="left", style="bold white")
    table.add_column("Max Timestamp", justify="left", style="bold white")

    # Loop through each file in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".parquet"):
            file_path = os.path.join(folder_path, file_name)

            # Load the Parquet file
            df = pl.read_parquet(file_path)

            # Get file name without the .parquet extension
            file_base_name = file_name.replace('.parquet', '')

            # Get the number of rows, min timestamp, and max timestamp
            num_rows = df.height
            min_timestamp = df["transit_timestamp"].min()
            max_timestamp = df["transit_timestamp"].max()

            # Add a row to the table
            table.add_row(file_base_name, str(num_rows), str(min_timestamp), str(max_timestamp))

    # Print the table
    console.print(table)

# Example usage
folder_path = '/home/christianocean/mta/datatest/nyc_subway_hourly_ridership_partitioned'
analyze_parquet_folder(folder_path)


                           Parquet File Summary                           
┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ File Name ┃ Number of Rows ┃ Min Timestamp       ┃ Max Timestamp       ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ 2024_04   │        2160264 │ 2024-04-01 00:00:00 │ 2024-04-30 23:00:00 │
│ 2024_02   │        2050469 │ 2024-02-01 00:00:00 │ 2024-02-29 23:00:00 │
│ 2022_07   │        2052523 │ 2022-07-01 00:00:00 │ 2022-07-31 23:00:00 │
│ 2023_03   │        2179474 │ 2023-03-01 00:00:00 │ 2023-03-31 23:00:00 │
│ 2022_03   │        2104114 │ 2022-03-01 00:00:00 │ 2022-03-31 23:00:00 │
│ 2022_11   │        2061942 │ 2022-11-01 00:00:00 │ 2022-11-30 23:00:00 │
│ 2023_11   │        2128966 │ 2023-11-01 00:00:00 │ 2023-11-30 23:00:00 │
│ 2022_04   │        2031557 │ 2022-04-01 00:00:00 │ 2022-04-30 23:00:00 │
│ 2024_01   │        2186658 │ 2024-01-01 00:00:00 │ 2024-01-31 23:00:00 │
│ 2024_10   │              9 │ 2024-10-01 00:00:00 │ 2024-10-01 02:00:00 │
│ 2022_12   │        2129938 │ 2022-12-01 00:00:00 │ 2022-12-31 23:00:00 │
│ 2023_04   │        2098133 │ 2023-04-01 00:00:00 │ 2023-04-30 23:00:00 │
│ 2022_08   │        2077164 │ 2022-08-01 00:00:00 │ 2022-08-31 23:00:00 │
│ 2023_01   │        2147754 │ 2023-01-01 00:00:00 │ 2023-01-31 23:00:00 │
│ 2023_10   │        2204295 │ 2023-10-01 00:00:00 │ 2023-10-31 23:00:00 │
│ 2022_05   │        2095321 │ 2022-05-01 00:00:00 │ 2022-05-31 23:00:00 │
│ 2023_08   │        2162687 │ 2023-08-01 00:00:00 │ 2023-08-31 23:00:00 │
│ 2023_12   │        2195693 │ 2023-12-01 00:00:00 │ 2023-12-31 23:00:00 │
│ 2023_07   │        2125938 │ 2023-07-01 00:00:00 │ 2023-07-31 23:00:00 │
│ 2023_09   │        2087132 │ 2023-09-01 00:00:00 │ 2023-09-30 23:00:00 │
│ 2023_02   │        1942348 │ 2023-02-01 00:00:00 │ 2023-02-28 23:00:00 │
│ 2024_07   │        2160996 │ 2024-07-01 00:00:00 │ 2024-07-31 23:00:00 │
│ 2024_08   │        2195972 │ 2024-08-01 00:00:00 │ 2024-08-31 23:00:00 │
│ 2023_06   │        2122984 │ 2023-06-01 00:00:00 │ 2023-06-30 23:00:00 │
│ 2022_09   │        2021999 │ 2022-09-01 00:00:00 │ 2022-09-30 23:00:00 │
│ 2024_06   │        2152096 │ 2024-06-01 00:00:00 │ 2024-06-30 23:00:00 │
│ 2022_02   │        1792341 │ 2022-02-01 00:00:00 │ 2022-02-28 23:00:00 │
│ 2024_05   │        2242532 │ 2024-05-01 00:00:00 │ 2024-05-31 23:00:00 │
│ 2024_09   │        2303124 │ 2024-09-01 00:00:00 │ 2024-09-30 23:00:00 │
│ 2022_10   │        2097211 │ 2022-10-01 00:00:00 │ 2022-10-31 23:00:00 │
│ 2024_03   │        2209152 │ 2024-03-01 00:00:00 │ 2024-03-31 23:00:00 │
│ 2023_05   │        2192627 │ 2023-05-01 00:00:00 │ 2023-05-31 23:00:00 │
│ 2022_06   │        2050052 │ 2022-06-01 00:00:00 │ 2022-06-30 23:00:00 │
└───────────┴────────────────┴─────────────────────┴─────────────────────┘